In [8]:
import os


def interval_mapping(image, from_min, from_max, to_min, to_max):
    from_range = from_max - from_min
    to_range = to_max - to_min
    scaled = np.array((image - from_min) / float(from_range), dtype=float)
    return to_min + (scaled * to_range)

def load_videos_path(dataset_path):
    videos_path = []
    
    for root, subdir, filesname in os.walk(dataset_path):
        for filename in filesname:
            if 'training/' in root and '.mp4' in filename:
                videos_path.append(os.path.join(root, filename))
    
    return videos_path


dataset_path = 'dataset'
videos_path = load_videos_path(dataset_path)

6451


In [9]:
import numpy as np
import cv2


def progress(current, until):
    p = int(current / until * 100)
    return '[%s%s%s] %d/%d' % (p * '=', '>' if current else ' ', (99-p) * ' ', current, until)

def show_progress(current, until):
    print(progress(current, until), end='\r')

def read_video(video_path, size=(480, 640, 3)):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    frames = []

    while ret:
        if frame.shape != size:
            frame = cv2.resize(frame, (size[1], size[0]))
        frames.append(frame)
        ret, frame = cap.read()

    return np.array(frames)

def calc_mean(videos_path, size=(480, 640, 3), verbose=True):
    acc = np.array([0.0, 0.0, 0.0])
    count = 0

    if verbose:
        print('calculando a media...')
    
    for index, path in enumerate(videos_path):
        try:
            if verbose:
                show_progress(index, len(videos_path))
            frames = read_video(path, size)
            acc += np.sum(frames, axis=(0, 1, 2))
            count += frames.shape[0] # numero de imagens por video
        except:
            print('calc_mean: Nao foi possivel processar %s.' % path)
    
    if verbose:
        show_progress(len(videos_path), len(videos_path))
        print()
    
    acc /= count * size[0] * size[1]
    return acc

def calc_std(mean, videos_path, size=(480, 640, 3), verbose=True):
    std = 0.0
    n = 0
    
    if verbose:
        print('calculando o desvio padrao...')
    
    for index, path in enumerate(videos_path):
        try:
            if verbose:
                show_progress(index, len(videos_path))
            frames = read_video(path, size)
            count = len(frames) * size[0] * size[1]
            std += (np.sum(frames, axis=(0, 1, 2)) / count - mean) ** 2
            n += 1
        except:
            print('calc_std: Nao foi possivel processar %s.' % path)
    
    if verbose:
        show_progress(len(videos_path), len(videos_path))
        print()
    
    return (std/(n-1)) ** (1/2)


size = (240, 320, 3)
mean = calc_mean(videos_path, size=size)
std = calc_std(mean, videos_path, size=size)

calculando a media...
[====================================================================================================>] 6451/6451
calculando o desvio padrao...
[====================================================================================================>] 6451/6451


In [11]:
print('mean:', mean)
print('standard deviation:', std)

mean: [135.43380275 145.99755187  87.54378712]
standard deviation: [46.61698649 51.50298563 44.30687373]


In [12]:
mean = np.array([130.94048136, 141.99215166,  85.33310485])
std = np.array([47.61450582, 52.96276823, 44.66200051])